# Урок 3. 

## Домашнее задание

Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred_proba.

Создайте функцию calc_pred, возвращающую предсказанный класс. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred.

*Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score. (по желанию)

Могла ли модель переобучиться? Почему?

In [219]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)

In [220]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [221]:
X

array([[   1.,    1.,  500.,    1.],
       [   1.,    1.,  700.,    1.],
       [   1.,    2.,  750.,    2.],
       [   1.,    5.,  600.,    1.],
       [   1.,    3., 1450.,    2.],
       [   1.,    0.,  800.,    1.],
       [   1.,    5., 1500.,    3.],
       [   1.,   10., 2000.,    3.],
       [   1.,    1.,  450.,    1.],
       [   1.,    2., 1000.,    2.]])

In [222]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

In [223]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [224]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [225]:
X_st

array([[ 1.        ,  1.        , -0.97958969,  1.        ],
       [ 1.        ,  1.        , -0.56713087,  1.        ],
       [ 1.        ,  2.        , -0.46401617,  2.        ],
       [ 1.        ,  5.        , -0.77336028,  1.        ],
       [ 1.        ,  3.        ,  0.97958969,  2.        ],
       [ 1.        ,  0.        , -0.36090146,  1.        ],
       [ 1.        ,  5.        ,  1.08270439,  3.        ],
       [ 1.        , 10.        ,  2.11385144,  3.        ],
       [ 1.        ,  1.        , -1.08270439,  1.        ],
       [ 1.        ,  2.        ,  0.05155735,  2.        ]])

## Изменим функцию calc_logloss так, чтобы нули не попадали в np.log

In [226]:
def calc_logloss(y, y_pred):
    y_pred = y_pred.astype(np.float64)
    y_pred[y_pred == 0] = 0.000001   #меняем все 0 на близкое к 0 значение
    y_pred[y_pred == 1] = 0.999999   #меняем все 1 на близкое к 1 значение
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [227]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.8, 0.1])
calc_logloss(y1, y_pred1)

0.164252033486018

In [228]:
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0]) 
calc_logloss(y1, y_pred1)

1.000000500029089e-06

In [229]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [230]:
z = np.linspace(-10, 10, 101)

In [231]:
probabilities = sigmoid(z)

## Подберем аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным. ##



Количество итераций будем подбирать за счёт введния разницы ошибки, при которой алгоритм будет останавливаться, а величину шага спуска будем подбирать из массива, который создадим сами:

In [232]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    min_err_dist = 1e-6     #вводим разницу ошибок, при которой алгоритм останавливается
    err_dist = np.inf
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if err_dist - err < min_err_dist:
            break
           
        err_dist = err
        
    return err, W, i

In [233]:
eta_array = [10, 1, 0.1, 0.01, 0.001, 0.0001]
for j in range(len(eta_array)):
    err, W, i = eval_model(X_st, y, iterations=100000, eta=eta_array[j])
    print(f'log los = {err}    iterations={i}   W={W}   eta = {eta_array[j]}')

log los = 24.806989015780175    iterations=1   W=[ 1.70233018 -1.6470242   0.94120115  5.86105201]   eta = 10
log los = 1.288059673857334    iterations=1   W=[ 0.41375036 -0.49353691  0.76427729  1.6960345 ]   eta = 1
log los = 0.1450623872289063    iterations=48798   W=[-24.24175175  -2.56849582  -6.38441957  20.08146297]   eta = 0.1
log los = 0.2644635503438669    iterations=55058   W=[-7.85348807 -1.16591973 -1.51033426  6.85770908]   eta = 0.01
log los = 0.4113251105511603    iterations=56628   W=[-1.51045506 -0.92058405  0.76776984  2.49708788]   eta = 0.001
log los = 0.5114992447814185    iterations=16725   W=[ 0.24318841 -0.71647905  0.70721001  1.27956669]   eta = 0.0001


Т.о. минимальное значение log loss будет при параметрах iterations=48798 и eta = 0,1

## Создадим функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [234]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [235]:
def calc_pred_proba(w, x): 
    pred_proba = sigmoid(np.dot(x, w))
    return pred_proba

In [236]:
err, W, i = eval_model(X_st, y, iterations=50000, eta=0.1)
W

array([-24.24175175,  -2.56849582,  -6.38441957,  20.08146297])

In [237]:
y_pred_proba = calc_pred_proba(W, X_st)

print(y_pred_proba)

[0.3835108  0.04277916 0.99999893 0.00000576 0.87670332 0.13515332
 0.99999991 0.0400007  0.54578501 0.9999712 ]


## Создадим функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [240]:
def calc_pred(w, x): 
    pred = sigmoid(np.dot(x, w))
    y_pred = []
    for i in range(len(pred)):
        if pred[i] <= 0.5:
            y_pred.append(0)
        else:
            y_pred.append(1)
    return y_pred

In [241]:
y_pred = calc_pred(W, X_st)
print(y_pred)

[0, 0, 1, 0, 1, 0, 1, 0, 1, 1]


## Могла ли модель переобучиться? Почему? 

Модель логистической регрессии склонна к переобучению, так как предсказания линейной модели преобразуем в вероятность с помощью сигмоиды. А сигмоида не имеет максимума и минимума - только асимптоты в 0 и 1, градиентный спуск не может достичь оптимального решения с помощью градиентных шагов доводя веса до все более экстремальных значений, пытаясь достичь нулевых потерь. При большой размерности данных вероятность этого еще больше увеличивается.